Ставим нужные библиотеки

In [ ]:
! pip install azureml-sdk[notebooks,automl] --ignore-installed

In [ ]:
! curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

In [ ]:
! az login

In [2]:
! az extension add -n azure-cli-ml

Extension 'azure-cli-ml' is already installed.


Создаем воркспейс

__NOTE__: название ресурс группы (ключ -g) и идентификатор подписки необходимо указывать свои, а не мои!

In [3]:
! az ml workspace create -w ml-workspace-1 -g hadoop2 --sku enterprise

Deploying KeyVault with name mlworkspkeyvaulta02d3d7c.
Deploying StorageAccount with name mlworkspstorage700ac68a9.
Deployed KeyVault with name mlworkspkeyvaulta02d3d7c. Took 23.31 seconds.
Deployed StorageAccount with name mlworkspstorage700ac68a9. Took 25.33 seconds.
Deploying AppInsights with name mlworkspinsightsd4af882e.
Deployed AppInsights with name mlworkspinsightsd4af882e. Took 50.74 seconds.
Deploying Workspace with name ml-workspace-1.
Deployed Workspace with name ml-workspace-1. Took 23.06 seconds.
{
  "applicationInsights": "/subscriptions/ddb191b9-4c87-419a-ab15-ed1e8a3999c2/resourcegroups/hadoop2/providers/microsoft.insights/components/mlworkspinsightsd4af882e",
  "creationTime": "2020-03-03T22:51:29.0640449+00:00",
  "description": "",
  "friendlyName": "ml-workspace-1",
  "hbiWorkspace": false,
  "id": "/subscriptions/ddb191b9-4c87-419a-ab15-ed1e8a3999c2/resourceGroups/hadoop2/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace-1",
  "identityPrincipalI

In [4]:
import azureml.core
from azureml.core.workspace import Workspace
ws = Workspace.get(
     name = "ml-workspace-1",
     subscription_id = "ddb191b9-4c87-419a-ab15-ed1e8a3999c2",
     resource_group = 'hadoop2')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [5]:
from azureml.core import Experiment
experiment = Experiment(workspace=ws, name='experiment-1')

Создаем кластер

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster
cluster_name = "cluster"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=2,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Creating
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Создаем директорию для эксперимента. В ней будут все необходимые данные для запуска. Как минимум скрипт для обучения.

In [9]:
! mkdir -p iris

In [10]:
%%writefile iris/train.py
import argparse
import os

import numpy as np

from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import joblib

from azureml.core.run import Run
run = Run.get_context()


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('--kernel', type=str, default='linear',
                        help='Kernel type to be used in the algorithm')
    parser.add_argument('--penalty', type=float, default=1.0,
                        help='Penalty parameter of the error term')

    args = parser.parse_args()
    run.log('Kernel type', np.str(args.kernel))
    run.log('Penalty', np.float(args.penalty))

    iris = datasets.load_iris()

    X = iris.data
    y = iris.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    from sklearn.svm import SVC
    svm_model_linear = SVC(kernel=args.kernel, C=args.penalty).fit(X_train, y_train)
    svm_predictions = svm_model_linear.predict(X_test)

    accuracy = svm_model_linear.score(X_test, y_test)
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(accuracy))
    run.log('Accuracy', np.float(accuracy))
    
    cm = confusion_matrix(y_test, svm_predictions)
    print(cm)

    os.makedirs('outputs', exist_ok=True)
    joblib.dump(svm_model_linear, 'outputs/model.joblib')


if __name__ == '__main__':
    main()

Overwriting iris/train.py


In [11]:
from azureml.train.sklearn import SKLearn

script_params = {
    '--kernel': 'linear',
    '--penalty': 1.0,
}

estimator = SKLearn(
    source_directory='iris', 
    script_params=script_params,
    compute_target=cluster,
    entry_script='train.py',
    pip_packages=['joblib==0.13.2']
)

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib']. We cannot guarantee image build will succeed.


Запускаем эксперимент

In [13]:
run = experiment.submit(estimator)
run.wait_for_completion(show_output = True)

RunId: experiment-1_1583276317_762433cd
Web View: https://ml.azure.com/experiments/experiment-1/runs/experiment-1_1583276317_762433cd?wsid=/subscriptions/ddb191b9-4c87-419a-ab15-ed1e8a3999c2/resourcegroups/hadoop2/workspaces/ml-workspace-1

Streaming azureml-logs/55_azureml-execution-tvmps_f170782aa88113db359ff118856efb49a5bf74a768fb174ca83fff677c126106_d.txt

2020-03-03T22:58:55Z Starting output-watcher...
2020-03-03T22:58:55Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_27eeb2b1023e546dd95d4eb4d3675de0
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
14886f7a730e: Pulling fs layer
ca5c52be38b9: Pulling 

{'runId': 'experiment-1_1583276317_762433cd',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-03-03T22:58:50.04023Z',
 'endTimeUtc': '2020-03-03T23:00:48.965364Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'e14ac3c8-2281-4231-b4bf-a247e9c38e38',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': ['--kernel', 'linear', '--penalty', '1.0'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cluster',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment experiment-1 Environment',
   'version': 'Autosave_2020-03-03T22:57:30Z_bee33773',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'co

После успешного завершения, регистрируем полученную модель

In [14]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='sklearn-iris', 
                           model_path='outputs/model.joblib',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.20.3',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

Развертываем модель в облаке

In [15]:
web_service = Model.deploy(ws, "scikit-learn-service-iris", [model])
web_service.wait_for_deployment(show_output=True)

Running..........................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


Проверяем работу

In [17]:
print(web_service.scoring_uri)

http://d166e099-1e70-4e51-bfb9-c48607983979.westus2.azurecontainer.io/score


In [18]:
import requests

In [21]:
requests.post(web_service.scoring_uri, json={
    'method': 'predict',
    'data': [5.1, 3.5 ,1.4, 0.2]
}).json()

{'predict': 0}

In [23]:
requests.post(web_service.scoring_uri, json={
    'method': 'predict',
    'data': [6.7,3.1,4.4,1.4]
}).json()

{'predict': 1}

In [22]:
requests.post(web_service.scoring_uri, json={
    'method': 'predict',
    'data': [7.7,2.8,6.7,2.0]
}).json()

{'predict': 2}

Для того, чтобы полноценно утилизировать кластер в Ажуре, можно искать лучшую модель, подбирая гиперпараметры. Для этого в Ажуре есть специальная обертка для эксперимента. Вот пример того, как можно воспользоваться этой оберткой.
```

from azureml.train.hyperdrive.runconfig import HyperDriveRunConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    

param_sampling = RandomParameterSampling( {
    "--kernel": choice('linear', 'rbf'),
    "--penalty": choice(0.5, 1)
    }
)

hyperdrive_run_config = HyperDriveRunConfig(estimator=estimator,
                                            hyperparameter_sampling=param_sampling, 
                                            primary_metric_name='Accuracy',
                                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs=12,
                                            max_concurrent_runs=4)
hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True)

best_run = hyperdrive_run.get_best_run_by_primary_metric()
model = best_run.register_model(model_name='<model>', model_path='outputs/model.joblib')
```

Смотри https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriverunconfig?view=azure-ml-py и https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py

__Задача__:
* Поиграться с гиперпараметрами. Выбрать какое-то множество параметром для текущей модели (большее, чем в примере выше - добавить кернелей и пеналтей) и запустить расчеты в Azure ML. Лучшую из получившихся моделей задеплоить в облако и сделать запрос к ней через requests.

### Собираем такую же модель, но в визуальном редакторе

Далее речь пойдет о работе с визуальным редактором.

В семинарских заметках оставлю полезные ссылки.
* https://docs.microsoft.com/en-us/azure/machine-learning/
* https://github.com/Azure/MachineLearningNotebooks

__Задача__:
* Создать аналогичный пайплайн в графическом редакторе с перебором гиперпараметров. ( В качестве ответа - выполненный графа в ажур мл).